# Parameter Identification Demo
# Levitador Magnético Benchmark

This notebook demonstrates how to use the modular optimization framework to identify parameters of the levitator system.

## Overview

We'll demonstrate:
1. Loading the benchmark problem
2. Running individual optimizers
3. Comparing multiple algorithms
4. Visualizing results

In [ ]:
# Import required libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent) if 'notebooks' in str(Path.cwd()) else '.')

from levitador_benchmark import LevitadorBenchmark
from src.optimization import (
    RandomSearch, DifferentialEvolution, GeneticAlgorithm,
    GreyWolfOptimizer, ArtificialBeeColony, HoneyBadgerAlgorithm,
    ShrimpOptimizer, TianjiOptimizer
)
from src.visualization.plots import plot_convergence, plot_comparison_boxplot

print("✓ All imports successful")

## 1. Create Benchmark Problem

The benchmark represents the levitator system with parameters to identify: k0, k, and a.

In [ ]:
# Create benchmark instance
problema = LevitadorBenchmark(random_seed=42, verbose=True)

print(f"\nProblem dimension: {problema.dim}")
print(f"Search space bounds: {problema.bounds}")
print(f"Reference solution: {problema.reference_solution}")

## 2. Test a Single Optimizer

Let's start with Grey Wolf Optimizer as an example.

In [ ]:
# Run Grey Wolf Optimizer
print("Running Grey Wolf Optimizer...\n")
gwo = GreyWolfOptimizer(problema, pop_size=30, max_iter=50, random_seed=42, verbose=True)
best_solution, best_fitness = gwo.optimize()

print(f"\n🏆 Best Solution Found:")
print(f"  k0 = {best_solution[0]:.6f} H")
print(f"  k  = {best_solution[1]:.6f} H")
print(f"  a  = {best_solution[2]:.6f} m")
print(f"\nFitness (MSE): {best_fitness:.6e}")
print(f"Evaluations: {gwo.evaluations}")

## 3. Visualize Convergence

In [ ]:
# Plot convergence curve
plt.figure(figsize=(10, 6))
plt.plot(gwo.history, linewidth=2, color='blue')
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Best Fitness (MSE)', fontsize=12)
plt.title('Grey Wolf Optimizer - Convergence', fontsize=14, fontweight='bold')
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Compare Multiple Algorithms

Let's compare several bio-inspired algorithms on the same problem.

In [ ]:
# Define optimizers to compare
optimizers_to_test = [
    ('Random Search', RandomSearch, {'n_iterations': 1500, 'random_seed': 42, 'verbose': False}),
    ('Differential Evolution', DifferentialEvolution, {'pop_size': 30, 'max_iter': 50, 'random_seed': 42, 'verbose': False}),
    ('Genetic Algorithm', GeneticAlgorithm, {'pop_size': 30, 'generations': 50, 'random_seed': 42, 'verbose': False}),
    ('Grey Wolf', GreyWolfOptimizer, {'pop_size': 30, 'max_iter': 50, 'random_seed': 42, 'verbose': False}),
    ('Artificial Bee Colony', ArtificialBeeColony, {'pop_size': 30, 'max_iter': 50, 'random_seed': 42, 'verbose': False}),
    ('Honey Badger', HoneyBadgerAlgorithm, {'pop_size': 30, 'max_iter': 50, 'random_seed': 42, 'verbose': False}),
    ('Shrimp', ShrimpOptimizer, {'pop_size': 30, 'max_iter': 50, 'random_seed': 42, 'verbose': False}),
    ('Tianji', TianjiOptimizer, {'pop_size': 30, 'max_iter': 50, 'random_seed': 42, 'verbose': False}),
]

# Run all optimizers
results = {}
histories = {}

for name, optimizer_class, config in optimizers_to_test:
    print(f"Running {name}...")
    optimizer = optimizer_class(problema, **config)
    best_sol, best_fit = optimizer.optimize()
    
    results[name] = best_fit
    histories[name] = optimizer.history
    print(f"  ✓ Fitness: {best_fit:.6e}\n")

print("\n" + "="*60)
print("COMPARISON RESULTS")
print("="*60)
for name in sorted(results.keys(), key=lambda x: results[x]):
    print(f"{name:<25} {results[name]:.6e}")

## 5. Visualize All Convergence Curves

In [ ]:
# Plot all convergence curves
plot_convergence(histories, title='Algorithm Convergence Comparison', log_scale=True)

## 6. Statistical Comparison (Multiple Trials)

For a more robust comparison, let's run multiple trials of each algorithm.

In [ ]:
# Run multiple trials
n_trials = 10
algorithms_to_compare = [
    ('DE', DifferentialEvolution, {'pop_size': 30, 'max_iter': 50, 'verbose': False}),
    ('Grey Wolf', GreyWolfOptimizer, {'pop_size': 30, 'max_iter': 50, 'verbose': False}),
    ('ABC', ArtificialBeeColony, {'pop_size': 30, 'max_iter': 50, 'verbose': False}),
    ('Honey Badger', HoneyBadgerAlgorithm, {'pop_size': 30, 'max_iter': 50, 'verbose': False}),
]

multi_trial_results = {}

for name, optimizer_class, config in algorithms_to_compare:
    print(f"Running {name} ({n_trials} trials)...")
    trial_results = []
    
    for trial in range(n_trials):
        # Use different seed for each trial
        config['random_seed'] = 42 + trial
        optimizer = optimizer_class(problema, **config)
        _, fitness = optimizer.optimize()
        trial_results.append(fitness)
    
    multi_trial_results[name] = trial_results
    mean_fitness = np.mean(trial_results)
    std_fitness = np.std(trial_results)
    print(f"  Mean: {mean_fitness:.6e} ± {std_fitness:.6e}\n")

In [ ]:
# Plot box plot comparison
plot_comparison_boxplot(
    multi_trial_results,
    title=f'Algorithm Comparison ({n_trials} trials)',
    log_scale=True
)

## 7. Analyze Best Solution

Let's examine the best solution found and compare it to the reference.

In [ ]:
# Get the best algorithm from single trial
best_algorithm = min(results.keys(), key=lambda x: results[x])
print(f"Best performing algorithm: {best_algorithm}")
print(f"Fitness: {results[best_algorithm]:.6e}")

# Compare with reference
print(f"\nReference solution:")
print(f"  k0 = {problema.reference_solution[0]:.6f} H")
print(f"  k  = {problema.reference_solution[1]:.6f} H")
print(f"  a  = {problema.reference_solution[2]:.6f} m")

ref_fitness = problema.fitness_function(problema.reference_solution)
print(f"\nReference fitness: {ref_fitness:.6e}")

## 8. Summary and Recommendations

Based on the results:

1. **Best Performance**: The algorithm with lowest MSE provides the most accurate parameter identification
2. **Convergence Speed**: Check which algorithms converge fastest
3. **Reliability**: Algorithms with lower standard deviation across trials are more reliable
4. **Computational Cost**: Consider evaluations/runtime for practical applications

### Using the Framework

To use this framework in your own code:

```python
from levitador_benchmark import LevitadorBenchmark
from src.optimization import GreyWolfOptimizer

# Create problem
problema = LevitadorBenchmark()

# Run optimizer
optimizer = GreyWolfOptimizer(problema, pop_size=30, max_iter=100)
best_solution, best_fitness = optimizer.optimize()

print(f"Solution: {best_solution}")
print(f"Fitness: {best_fitness}")
```

Or use the command-line benchmark runner:

```bash
python scripts/run_benchmark.py --config config/default.yaml
```